#### To do:
- get names from file 
- access dbpedia query 
- extract information

#### To do after:
- info extraction from comments 
-- maybe only when infomation is missing

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import rdflib

In [2]:
pop_ents = []
sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-186')
sparql.setReturnFormat(JSON)
query = f"""
        SELECT ?s
        WHERE {{
        ?s a <http://example.com/popularEntity> .
            }}
            """
    #print(subject)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
qres = sparql.query().convert() 
for r in qres["results"]["bindings"]:
    pop_ents.append(r["s"]["value"])
len(pop_ents), pop_ents

(186,
 ['http://yago-knowledge.org/resource/David_Bowie',
  'http://yago-knowledge.org/resource/Kanye_West',
  'http://yago-knowledge.org/resource/Eminem',
  'http://yago-knowledge.org/resource/Jennifer_Aniston',
  'http://yago-knowledge.org/resource/George_W._Bush',
  'http://yago-knowledge.org/resource/Prince_(musician)',
  'http://yago-knowledge.org/resource/Justin_Bieber',
  'http://yago-knowledge.org/resource/Drake_(musician)',
  'http://yago-knowledge.org/resource/Heath_Ledger',
  'http://yago-knowledge.org/resource/John_Krasinski',
  'http://yago-knowledge.org/resource/Val_Kilmer',
  'http://yago-knowledge.org/resource/American_Horror_Story',
  'http://yago-knowledge.org/resource/Brad_Pitt',
  'http://yago-knowledge.org/resource/Japan',
  'http://yago-knowledge.org/resource/Barack_Obama',
  'http://yago-knowledge.org/resource/Bill_Clinton',
  'http://yago-knowledge.org/resource/Donald_Trump',
  'http://yago-knowledge.org/resource/Jimmy_Carter',
  'http://yago-knowledge.org/resou

In [3]:
s = []
s_source = []
for i in pop_ents:
    topic = i.split('/')[-1]
    split = topic.split('_')
    s.append(' '.join(split))
    
data = {'Entity': pop_ents, 'Label': s}
database=pd.DataFrame(data)
database

,Entity,Label
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West
2,http://yago-knowledge.org/resource/Eminem,Eminem
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush
...,...,...
181,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi
182,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh
183,http://yago-knowledge.org/resource/Maharashtra,Maharashtra
184,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.


In [4]:
len(database)

186

In [5]:
def query(popent):
    fun = False
    while fun == False:
        try:
            sparql = SPARQLWrapper("http://dbpedia.org/sparql")
            sparql.setReturnFormat(JSON)
            sparql.setQuery("""
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                PREFIX dbo: <http://dbpedia.org/ontology/>
                SELECT ?abstract
                WHERE { 
                ?s rdfs:label "%s" @en .
                ?s dbo:abstract ?abstract .
                FILTER (LANG(?abstract) = "en")
                }
                """ %(popent))
            results = sparql.query().convert()
            fun = True
        except:
            fun = False
    return results 

abstracts = []
j = 0
for label in database.Label:
    if j % 1000 == 0:
        print(j)
    pastablen = len(abstracts)
    results = query(label)
    if len(results["results"]["bindings"]) == 0:
        abstracts.append('')
        j+=1
        continue 
    for result in results["results"]["bindings"]:
        abstracts.append(result['abstract']['value'])
        j += 1

0


In [6]:
j, len(database)

(186, 186)

In [7]:
len(database), len(abstracts), len(database.Label)

(186, 186, 186)

In [8]:
database['Abstracts']= abstracts
database

,Entity,Label,Abstracts
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A..."
...,...,...,...
181,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...
182,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te..."
183,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...
184,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...


In [16]:
blanks = 0
notblanks = []
for ent,label,ab in database.values:
    if len(ab) == 0:
        print(label)
        blanks += 1
        print(ab)
    else:
        notblanks.append({'Entity': ent, 'Label': label, 'Abstract': ab})
#'Number of entites with no abstract:',blanks
data = pd.DataFrame(notblanks)
data

Charles, Prince of Wales

DMX (rapper)



,Entity,Label,Abstract
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A..."
...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...


In [15]:
data.to_csv('Abstracts-186.csv')